In [40]:
import pandas as pd
import torch
from tqdm.notebook import tqdm
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
EMBEDDING_MODEL = 'thenlper/gte-large'
DELIMITER=" "
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 128
K = 1000


## Datasets

Uncomment the dataset you want work on.

In [ ]:

# # ml-1m
# def ml_preprocessing(title):
#     title = " ".join(title.split(" ")[:-1]).strip()
#     if title.endswith(", The"):
#         title = "The " + title[:-5] 
#     if title.endswith(", A"):
#         title = "A " + title[:-3] 
#     return title

# data_path = "ML-1M/ml-1m.txt"
# titles_path = "ML-1M/ml-1m-titles.txt"
# title_freq_path = "ML-1M/ml-1m-train_item_freq.txt"
# similarity_indices_out = f"ML-1M/ml-1m-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"ML-1M/ml-1m-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# embeddings_out = f"ML-1M/ml-1m-embeddings-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = "ML-1M/ml-1m_timestamp.txt"
# preprocessing_title = ml_preprocessing


# ok

In [8]:
# Beauty
# data_path = "Beauty/Beauty.txt"
# titles_path = "Beauty/Beauty-titles.txt"
# title_freq_path = "Beauty/Beauty-train_item_freq.txt"
# similarity_indices_out = f"Beauty/Beauty-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"Beauty/Beauty-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = f"Beauty/Beauty-{EMBEDDING_MODEL.replace('/','_')}_timestamp.txt"
# preprocessing_title = lambda t: t


# ok

In [10]:
# # Tools
# data_path = "Tools/Tools.txt"
# titles_path = "Tools/Tools-titles.txt"
# title_freq_path = "Tools/Tools-train_item_freq.txt"
# similarity_indices_out = f"Tools/Tools-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"Tools/Tools-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = "Tools/Tools_timestamp.txt"
# preprocessing_title = lambda t: t


# ok

In [12]:
# # HomeKitchen
# data_path = "HomeKitchen/HomeKitchen.txt"
# titles_path = "HomeKitchen/HomeKitchen-titles.txt"
# title_freq_path = "HomeKitchen/HomeKitchen-train_item_freq.txt"
# similarity_indices_out = f"HomeKitchen/HomeKitchen-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"HomeKitchen/HomeKitchen-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = "HomeKitchen/HomeKitchen_timestamp.txt"
# preprocessing_title = lambda t: t


# ok

In [14]:
# # Pet Supplies
# data_path = "PetSupplies/Pet.txt"
# titles_path = "PetSupplies/Pet-titles.txt"
# title_freq_path = "PetSupplies/Pet-train_item_freq.txt"
# similarity_indices_out = f"PetSupplies/Pet-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"PetSupplies/Pet-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = "PetSupplies/Pet_timestamp.txt"
# preprocessing_title = lambda t: t


# ok

In [44]:
# # Steam
# data_path = "Steam/steam.txt"
# titles_path = "Steam/steam-titles.txt"
# title_freq_path = "Steam/steam-train_item_freq.txt"
# similarity_indices_out = f"Steam/steam-similarity-indices-{EMBEDDING_MODEL.replace('/','_')}.pt"
# similarity_values_out = f"Steam/steam-similarity-values-{EMBEDDING_MODEL.replace('/','_')}.pt"
# timestamp_path = f"Steam/steam_timestamp.txt"
# preprocessing_title = lambda t: t


# ok

## Calcualte Similarities

In [47]:
def sentence_transformer(model_name, batch_size, device):
    model = SentenceTransformer(model_name, device=device)
    def embed(sentences):
        embeddings = []
        batches = [sentences[x:x+batch_size] for x in range(0, len(sentences), batch_size)]
        for batch in tqdm(batches):
            embeddings.append(model.encode(batch, convert_to_numpy=False, convert_to_tensor=True))
        return torch.cat(embeddings, dim=0)
    return embed

embedding_func = sentence_transformer(model_name=EMBEDDING_MODEL, batch_size=batch_size, device=device)

In [49]:
titles_df = pd.read_csv(titles_path, names=['id', 'title'], delimiter=DELIMITER, escapechar="\\",encoding='latin-1') 
titles_df
#tive que adicionar o encodin latin-1 para resolver um erro de leitura no pandas pra arquivos tipo utf8(UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 829: invalid continuation byte)

,id,title
0,0,No Man's Sky
1,1,Defend the Highlands: World Tour
2,2,Titanis
3,3,Super Indie Karts
4,4,MechWarrior Onlineâ¢
...,...,...
13040,13040,Railroad Pioneer
13041,13041,Red Baron Pack
13042,13042,The Tomorrow War
13043,13043,Coffee Run


In [51]:
id_to_freq_df = pd.read_csv(title_freq_path, names=['id', 'freq'], delimiter=DELIMITER)
id_to_freq_series = pd.Series(id_to_freq_df.freq.values, index=id_to_freq_df.id)
id_to_freq = id_to_freq_series.to_dict()
titles_df['freq'] = id_to_freq_series
titles_df = titles_df[['id', 'freq', 'title']]
titles_df

,id,freq,title
0,0,2344,No Man's Sky
1,1,5,Defend the Highlands: World Tour
2,2,5,Titanis
3,3,51,Super Indie Karts
4,4,1241,MechWarrior Onlineâ¢
...,...,...,...
13040,13040,22,Railroad Pioneer
13041,13041,11,Red Baron Pack
13042,13042,18,The Tomorrow War
13043,13043,11,Coffee Run


In [ ]:
titles_df['title'] = titles_df['title'].apply(np.vectorize(preprocessing_title))
titles_df

,id,freq,title
0,0,2344,No Man's Sky
1,1,5,Defend the Highlands: World Tour
2,2,5,Titanis
3,3,51,Super Indie Karts
4,4,1241,MechWarrior Onlineâ¢
...,...,...,...
13040,13040,22,Railroad Pioneer
13041,13041,11,Red Baron Pack
13042,13042,18,The Tomorrow War
13043,13043,11,Coffee Run


In [ ]:

titles_list = titles_df['title'].tolist()
titles_embeddings = embedding_func(titles_list)
titles_embeddings

  0%|          | 0/102 [00:00<?, ?it/s]

tensor([[-0.0004,  0.0103, -0.0106,  ..., -0.0378, -0.0367, -0.0187],
        [-0.0043,  0.0107, -0.0173,  ..., -0.0336,  0.0030, -0.0124],
        [-0.0121,  0.0023,  0.0209,  ...,  0.0013, -0.0111, -0.0271],
        ...,
        [ 0.0198, -0.0089, -0.0265,  ..., -0.0272, -0.0248, -0.0182],
        [ 0.0059,  0.0021, -0.0147,  ...,  0.0027,  0.0039,  0.0062],
        [ 0.0249,  0.0242, -0.0204,  ..., -0.0394, -0.0184, -0.0256]])

In [57]:
titles_embeddings.shape

torch.Size([13045, 1024])

In [59]:
def get_similarity_matrix(emebddings, eps=1e-8, top_k=None):
    embeddings_norm = emebddings.norm(dim=1).unsqueeze(dim=1)  # (num_embeddings, 1)
    embeddings_normalized = emebddings /  torch.max(embeddings_norm, eps * torch.ones_like(embeddings_norm))
    if top_k is None:
        similarity_values = embeddings_normalized @ embeddings_normalized.T
        # fix numerical percison issues - where similarity_matrix[i,i] < similarity_matrix[i, k != i]
        similarity_values += torch.diag(torch.full((similarity_values.shape[0],), 1e-7, device=device))
        similarity_indices = torch.arange(similarity_values.shape[0]).unsqueeze(dim=0).repeat(similarity_values.shape[0], 1)

    else:
        n_embeddings = emebddings.shape[0]
        chunks = n_embeddings // 1000
        value_list = []
        indices_list = []
        for chunk in embeddings_normalized.chunk(chunks):
            similarity_out = chunk @ embeddings_normalized.T 
            values, indices = torch.topk(similarity_out, dim= -1, k=top_k, sorted=True)
            value_list.append(values)
            indices_list.append(indices)
        similarity_values = torch.cat(value_list, dim=0)
        similarity_indices = torch.cat(indices_list, dim=0)

    return similarity_values, similarity_indices

In [61]:
similarity_values, similarity_indices = get_similarity_matrix(titles_embeddings, top_k=K)
print(similarity_indices)
similarity_values

tensor([[    0,  8953, 11802,  ...,  2278, 11444,  1301],
        [    1,  6077, 11666,  ..., 12653,  6114,   953],
        [    2, 12764,  2042,  ...,  5585,  7243,  4694],
        ...,
        [13042, 11929,  8186,  ...,  6941,   342,  3452],
        [13043, 10547,  9832,  ...,  5576, 12228,  6094],
        [13044,  2480, 11764,  ...,  3351,  5030, 11177]])


tensor([[1.0000, 0.8636, 0.8624,  ..., 0.8048, 0.8048, 0.8048],
        [1.0000, 0.9304, 0.8697,  ..., 0.7795, 0.7795, 0.7795],
        [1.0000, 0.8744, 0.8615,  ..., 0.7871, 0.7871, 0.7871],
        ...,
        [1.0000, 0.9324, 0.8616,  ..., 0.7937, 0.7937, 0.7937],
        [1.0000, 0.8906, 0.8746,  ..., 0.7719, 0.7719, 0.7718],
        [1.0000, 0.9088, 0.8871,  ..., 0.8033, 0.8033, 0.8032]])

Save all embeddings and similarities

In [64]:
torch.save(similarity_indices, similarity_indices_out)
torch.save(similarity_values, similarity_values_out)

In [66]:
! echo `date +'%I_%M_%d_%m'` > {timestamp_path}